In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver as uc
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException 
import pandas as pd
from selenium.webdriver.common.by import By
import re

chargement de la page youtube + pas de détection comme étant rebot

In [10]:
if __name__ == "__main__":
    options = webdriver.ChromeOptions()
    options.add_argument('proxy-server=106.122.8.54:3128')
    browser = uc.Chrome(
        options=options,
    )
    browser.get("https://www.youtube.com/@SkyNews/videos")
    sleep(2)   

la fonction scroll

In [25]:
def scroll():
    while(True):
        height = browser.execute_script("return document.body.scrollHeight")
        sleep(1)
        browser.find_element_by_tag_name('body').send_keys(Keys.END)
        if int(height)==0:
            break

répeter le scrolling jusqu'arriver au nombre voulu des video (plus de 10000) et récupérer la liste des liens

In [ ]:
last_position = browser.execute_script("return window.pageYOffset ;")
while True:
    fin = True
    for i in range(3):
        scroll()
        sleep(2)
        curr_position = browser.execute_script("return window.pageYOffset ;")
        if (curr_position!=last_position):
            fin = False
            break
    l=browser.find_elements(by=By.XPATH, value='//ytd-rich-item-renderer[@class="style-scope ytd-rich-grid-row"]//a[@id="video-title-link"]')
    print(len(l))
    if fin or (len(l)>10000) :
        links=[i.get_attribute('href') for i in l]
        break
    else: last_position=curr_position
    

In [2]:
len(links)

10020

chargement de chaque video et récupération des informations souhaitées

In [ ]:
titles,durées,vues,dates_pub,likes,shares,chaine = [],[],[],[],[],[],[]
for link in links[8000:]:
    browser.get(link)
    sleep(3)
    try:
        titles.append(browser.find_element(by=By.XPATH, value='//div[@id="title"]/h1/yt-formatted-string').text)
        likes.append(browser.find_element(by=By.XPATH, value='//div[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button/div[2]/span').text)
        vues.append(browser.find_element(by=By.XPATH, value='//yt-formatted-string[@id="info"]/span[1]').text)
        dates_pub.append(browser.find_element(by=By.XPATH, value='//yt-formatted-string[@id="info"]/span[3]').text)
        durées.append(browser.find_element(by=By.XPATH, value='//div[@class="ytp-time-display notranslate"]//span[@class="ytp-time-duration"]').text)
        chaine.append(browser.find_elements(by=By.XPATH, value='//ytd-channel-name[@id="channel-name"]//a')[0].text)
        try:
            shares.append(browser.find_element(by=By.XPATH, value='//h2[@id="count"]/yt-formatted-string/span[1]').text)
        except:
            sleep(1)
            try:
                shares.append(browser.find_element(by=By.XPATH, value='//h2[@id="count"]/yt-formatted-string/span[1]').text)
            except:
                shares.append('')
    except : continue

data pre-processing

In [ ]:
df_final=pd.DataFrame({"title":titles , "chaine":chaine , "vues":vues,   "likes":likes , "comments":shares  , "durée":durées , "date_publication":dates_pub , "link":links})
df_final["durée"] =df_final['durée'].apply(lambda x: None if x == '' else x)
df_final["title"] =df_final['title'].apply(lambda x: None if x == '' else x)
df_final["vues"] =df_final['vues'].apply(lambda x: None if x == '' else x)
df_final["date_publication"] =df_final['date_publication'].apply(lambda x: None if x == '' else x)
df_final["chaine"] =df_final['chaine'].apply(lambda x: None if x == '' else x)
df_final["likes"] =df_final['likes'].apply(lambda x: None if x == '' else x)
df_final["comments"] =df_final['comments'].apply(lambda x: None if x == '' else x)

In [15]:
df_final.shape

(9627, 8)

In [7]:
df_finale=df_final

In [6]:
df_final.shape

(7134, 8)

In [14]:
df_final.to_csv("res.csv")